In [ ]:
import pandas as pd
import numpy as np
import re
import os

In [ ]:
# Inpust section

df = pd.read_excel(r"D:\coldwell\Commercial Website sheet.xlsx")
projct_table = pd.read_excel(r"D:\coldwell\ProjectTable_20231113173024.xlsx")

#export_at = r"D:\coldwell\pending"
#status__ = "pending"

export_at = r"D:\coldwell\Publish"
status__ = "published"

project_type = "commertial"
#project_type = "Resedetial"
#project_type = "Costal"


In [ ]:
                              ### ======================= MAPS ======================= ###

# Replaced all real values with dummy ones for confidentiality purposes

translation = {'English word 1': 'الاسم العربي 1', 'ُEnglish word 2': 'الاسم العربي 2' }
category_map = {'wrongcat.writingstyle': 'Right Cat. Writing Style', 'cat1sub2': 'Cat1', }
finishing_map = {'English word 1': 'الاسم العربي 1', 'ُEnglish word 2': 'الاسم العربي 2'}
Deliver_Status_map = {"inputsyle": "Unified Out Put Style", "inputsyle2": "Unified Out Put Style2" }
comparing_new_and_old_projects = {}

project_type_maps = {"commertial": {"dummyprojectname1": "Dummy Old Project Name 1.0", "dummyprojectnamephase5": "Dummy Project Name | Phase 5"},
                     "Resedetial": {"dummyresedetialprojectname1": "Dummy Resedetial Project Name 1.0", "dummyprojectnamephase5": "Dummy Old Project Name | Phase 5"},
                     "Costal":{"dummycostalprojectname1": "Dummy Costal Project Name 1.0", "dummyprojectcostalnamephase5": "Dummy Project Costal Name | Phase 5"}}

# Used to compare new naming styles with names already found on the system to be able to update them without considering them new
old_projects_naming_map =  project_type_maps[project_type]





                          ### ======================= Output Lists ======================= ###

Name,Starting_Price,Concept_Name,Area_From,Area_To,Finishing,Description,Is_Sold,Price_Per_Meter,Category,Project,Status,Import_Type,Name_AR,Description_AR,Finishing_AR,show_in_price_indicator= [],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]
Projects_list, Deliver_Statuses, Delivery_Dates, Down_Payment_list, Starting_Price_list, Installments_Years_list, Status_list = [], [], [], [], [], [], []
ucn_has_error = {}


                     ### ======================= Removing trash rows ======================= ###


no_ucn_df = df[df["Unit Concept Name"].isna()]
no_ucn_df["error"] = "NO UCN"
df = df[~df["Unit Concept Name"].isna()]

no_project_name_df = df[df["Project Name  ↑"].isna()]
no_project_name_df["error"] = "NO Project Name"
df = df[~df["Project Name  ↑"].isna()]

no_unit_type_df = df[df["Unit Type"].isna()]
no_unit_type_df["error"] = "No Unit Type"
df = df[~df["Unit Type"].isna()]

no_delivery_date_df = df[df["Delivery Date"].isna()]
no_delivery_date_df["error"] = "No Delivery Date"
df = df[~df["Delivery Date"].isna()]

catastrophic_errors = pd.concat([no_ucn_df, no_project_name_df,no_delivery_date_df,  no_unit_type_df], ignore_index=True, sort=False)
catastrophic_errors.to_excel(r"{}\{}".format(export_at,"catastrophic_errors.xlsx"), index=False)

                     ### ======================= Projects_Units_Sheet ======================= ###
for ucn in df["Unit Concept Name"].unique():

    error_note = ""

    # ----------- Name ----------- #

    try:
        Name.append(list(df[df["Unit Concept Name"] == ucn]["Unit Type"])[0])

        if list(df[df["Unit Concept Name"] == ucn]["Unit Type"])[0] is None:
            print(ucn, f"Name (Unit Type) is not found")
            error_note += f" |Name (Unit Type) is not found"
            Name.append(np.nan)


    except Exception as e:
        print(ucn, f": Name (Unit Type) has error {e}")
        error_note += f" |Name (Unit Type) has error {e}"
        Name.append(np.nan)

    # ----------- Project ----------- #

    try:
        Project.append(str(list(df[df["Unit Concept Name"] == ucn]["Project Name  ↑"])[0]) + " | " + str(list(df[df["Unit Concept Name"] == ucn]["Phase Name  ↑"])[0]))

        if list(df[df["Unit Concept Name"] == ucn]["Project Name  ↑"])[0] is None:
            print(ucn, f"Project Name not found")
            error_note += f" |Project Name not found"
            Project.append(np.nan)

        if list(df[df["Unit Concept Name"] == ucn]["Phase Name  ↑"])[0] is None:
            print(ucn, f"Phase Name not found")
            error_note += f" |Phase Name not found"
            Project.append(np.nan)

    except Exception as e:
        print(ucn, f": Project name or phase name has error {e}")
        error_note += f" |Project name or phase name has error {e}"
        Project.append(np.nan)

    # ----------- Starting_Price ----------- #

    try:
        Starting_Price.append(list(df[df["Unit Concept Name"] == ucn]["Price From"])[0])

        if list(df[df["Unit Concept Name"] == ucn]["Price From"])[0] is None:
            print(ucn, f": Price From is not found")
            error_note += f" |Price From is not found"
            Starting_Price.append(np.nan)

    except Exception as e:
        print(ucn, f"Price From has error {e}")
        error_note += f" |Price has error {e}"
        Starting_Price.append(np.nan)


    # ----------- Concept_Name (UCN) ----------- #

    Concept_Name.append(ucn)
    if ucn is None:
        print(ucn, f"Concept_Name (UCN) is not found")
        error_note += f" |Concept_Name (UCN) is not found"
        Concept_Name.append(np.nan)


    # ----------- Finishing ----------- #
    finishing_has_error = False
    try:
        Finishing.append(list(df[df["Unit Concept Name"] == ucn]["Finishing Type"])[0])

    except Exception as e:
        print(ucn, f"Finishing has error {e}")
        error_note += f" |Finishing has error {e}"
        Finishing.append(np.nan)
        finishing_has_error = True

    # ----------- Status & Import_Type ----------- #

    Status.append(status__)
    Import_Type.append("units")

    # ----------- Area_From ----------- #
    #check if the area has decimals or not
    try:
        if int(list(df[df["Unit Concept Name"] == ucn]["Build Up Area From"])[0]) == list(df[df["Unit Concept Name"] == ucn]["Build Up Area From"])[0]:
            Area_From.append(int(list(df[df["Unit Concept Name"] == ucn]["Build Up Area From"])[0]))
        else:
            Area_From.append(list(df[df["Unit Concept Name"] == ucn]["Build Up Area From"])[0])
    except:
        print(ucn, f":Area From is not found")
        error_note += f" |Area From is not found"
        Area_From.append(np.nan)


    # ----------- Area_To ----------- #
    #check if the area has decimals or not
    try:
        if int(list(df[df["Unit Concept Name"] == ucn]["Build Up Area To"])[0]) == list(df[df["Unit Concept Name"] == ucn]["Build Up Area To"])[0]:
            Area_To.append(int(list(df[df["Unit Concept Name"] == ucn]["Build Up Area To"])[0]))
        else:
            Area_To.append(list(df[df["Unit Concept Name"] == ucn]["Build Up Area To"])[0])
    except:
        Area_To.append(np.nan)


    # ----------- Category ----------- #
    try:
        Category.append(category_map[str(list(df[df["Unit Concept Name"] == ucn]["Unit Type"])[0]).lower().replace(" ","")])
    except:
        print(ucn, f"Category not found")
        error_note += f" |Category not found"
        Category.append(np.nan)


    # ----------- Finishing_AR ----------- #
    if not finishing_has_error:
        try:
            Finishing_AR.append(finishing_map[list(df[df["Unit Concept Name"] == ucn]["Finishing Type"])[0]])
        except:
            print(ucn, f":{list(df[df['Unit Concept Name'] == ucn]['Finishing Type'])[0]} Finishing transation not found")
            error_note += f" |{list(df[df['Unit Concept Name'] == ucn]['Finishing Type'])[0]} Finishing transation not found"
            Finishing_AR.append(np.nan)


    # ----------- Name_AR ----------- #
    try:
        Name_AR.append(translation[list(df[df["Unit Concept Name"] == ucn]["Unit Type"])[0]])
    except:
        print(ucn, f"Name transation not found")
        error_note += f" |Name transation not found"
        Name_AR.append(np.nan)

    # ----------- Append ----------- #
    if error_note != "":
        ucn_has_error[ucn] =error_note

    Description.append(np.nan)
    Is_Sold.append(0)
    Price_Per_Meter.append(0)
    Description_AR.append(np.nan)
    show_in_price_indicator = 1



                          ### ======================= Starting_price_Sheet ======================= ###

for project_name in df["Project Name  ↑"].unique():
    for phase in df[df["Project Name  ↑"] == project_name]["Phase Name  ↑"].unique():
        Projects_list.append(project_name+ " | "+ phase)
        mapped_status = Deliver_Status_map.get(re.sub(r'[^a-zA-Z0-9]', '', str(df[((df["Project Name  ↑"] == project_name) & (df["Phase Name  ↑"] == phase))]["Delivery Status"].iloc[0]).lower().replace(" ","")), np.nan)
        Deliver_Statuses.append(mapped_status)
        Delivery_Dates.append(f'1/1/{int(list(df[((df["Project Name  ↑"] == project_name) & (df["Phase Name  ↑"] == phase))]["Delivery Date"])[0])}')
        Down_Payment_list.append(min(list(df[((df["Project Name  ↑"] == project_name) & (df["Phase Name  ↑"] == phase))]["Down Payment"]))*100)
        Starting_Price_list.append(min(list(df[((df["Project Name  ↑"] == project_name) & (df["Phase Name  ↑"] == phase))]["Price From"])))
        Installments_Years_list.append(max(list(df[((df["Project Name  ↑"] == project_name) & (df["Phase Name  ↑"] == phase))]["Years of Installments"])))
        Status_list.append(status__)

                           ### ======================= Down_payments_Sheet ======================= ###

Down_payments = df[['Unit Concept Name', 'Down Payment', 'Years of Installments']].copy()
Down_payments = Down_payments.rename(columns = {'Unit Concept Name':'Concept Name', 'Down Payment': 'Downpayment', 'Years of Installments': 'Installments year'})
Down_payments.insert(2,"Sec Downpayment",np.nan)
Down_payments["Status"] = status__
Down_payments["Downpayment"] = Down_payments["Downpayment"].apply(lambda x: x*100)

# ----------- Check if the ucn has an error or not  ----------- #
if Down_payments["Installments year"].isnull().sum() >0 :

    for ucn in Down_payments[Down_payments["Installments year"].isnull()]["Concept Name"]:
        try:
            ucn_has_error[ucn] += " | Installments year is not found"
        except KeyError:
            ucn_has_error[ucn] = " | Installments year is not found"

if Down_payments["Downpayment"].isnull().sum() >0 :

    for ucn in Down_payments[Down_payments["Downpayment"].isnull()]["Concept Name"]:
        try:
            ucn_has_error[ucn] += " | Downpayment is not found"
        except KeyError:
            ucn_has_error[ucn] = " | Downpayment is not found"

Down_payments = Down_payments[Down_payments["Concept Name"].isin(ucn_has_error.keys()) == False]

                          ### ======================= Ceating data frames ======================= ###

output_df = pd.DataFrame({"Name": Name,"Starting Price": Starting_Price,"Concept Name": Concept_Name, "Area From": Area_From, "Area To": Area_To, "Finishing":Finishing ,"Description": Description,"Is Sold": Is_Sold, "Price Per Meter": Price_Per_Meter,"Category": Category,"Project": Project, "Status": Status,"Import Type": Import_Type ,"Name AR": Name_AR, "Description AR": Description_AR,"Finishing AR": Finishing_AR, "Show in price indicator": show_in_price_indicator})
has_errors_df = pd.DataFrame({"Concept Name": ucn_has_error.keys(), "Error\s":ucn_has_error.values()})
starting_price_df = pd.DataFrame({"Project": Projects_list, "Delivery Status": Deliver_Statuses,"Delivery Date": Delivery_Dates, "Down Payment": Down_Payment_list, "Starting Price": Starting_Price_list, "Installments Years":Installments_Years_list ,"Status": Status_list})

total = output_df.shape[0]  #<==== to check if the exported sheets are not missing any data


                ### ======================= Comparing New projects With old Projects ======================= ###

for index, row in projct_table.iterrows():
    old_projects_naming_map[re.sub(r'[^a-zA-Z0-9]', '', str(row['Name']).lower().replace(" ",""))] = row['Name']

output_df['Project'] = output_df['Project'].apply(lambda x: comparing_new_and_old_projects[re.sub(r'[^a-zA-Z0-9]', '', str(x).lower().replace(" ",""))] if re.sub(r'[^a-zA-Z0-9]', '', str(x).lower().replace(" ","")) in comparing_new_and_old_projects else x)
output_df['Project'] = output_df['Project'].apply(lambda x: old_projects_naming_map[re.sub(r'[^a-zA-Z0-9]', '', str(x).lower().replace(" ",""))] if re.sub(r'[^a-zA-Z0-9]', '', str(x).lower().replace(" ","")) in old_projects_naming_map else x)

starting_price_df['Project'] = starting_price_df['Project'].apply(lambda x: comparing_new_and_old_projects[re.sub(r'[^a-zA-Z0-9]', '', str(x).lower().replace(" ",""))] if re.sub(r'[^a-zA-Z0-9]', '', str(x).lower().replace(" ","")) in comparing_new_and_old_projects else x)
starting_price_df['Project'] = starting_price_df['Project'].apply(lambda x: old_projects_naming_map[re.sub(r'[^a-zA-Z0-9]', '', str(x).lower().replace(" ",""))] if re.sub(r'[^a-zA-Z0-9]', '', str(x).lower().replace(" ","")) in old_projects_naming_map else x)


                    ### ======================= Fltring and splitting data frames ======================= ###

# ----------- Check if the ucn has an error or not  ----------- #
has_errors_df = has_errors_df[has_errors_df["Concept Name"].isin(ucn_has_error.keys()) == True]
output_df = output_df[output_df["Concept Name"].isin(ucn_has_error.keys()) == False]


# ----------- Check if the project is already on the system or not  ----------- #
needs_to_be_added = output_df[~output_df.Project.isin(projct_table["Name"])]
output_df = output_df[output_df.Project.isin(projct_table["Name"])]


# ----------- Split down payment sheet depending on if the project is already on the system or not  ----------- #
Down_payments_to_upload = Down_payments[Down_payments["Concept Name"].isin(output_df["Concept Name"]) == True]
Down_payments_to_be_added = Down_payments[Down_payments["Concept Name"].isin(output_df["Concept Name"]) == False]

# ----------- Split starting price sheet depending on if the project is already on the system or not  ----------- #
starting_price_df_to_upload = starting_price_df[starting_price_df.Project.isin(output_df["Project"])]
starting_price_df_to_be_added = starting_price_df[~starting_price_df.Project.isin(output_df["Project"])]


                    ### ======================= Exporting Excel sheets ======================= ###

# Remove headers format from excel sheet
output_df.to_excel(r"{}\{}".format(export_at,"Coldwell ready_to_upload (units).xlsx"), index=False)
pd.io.formats.excel.ExcelFormatter.header_style = None  #<====== remove headers format from excel sheet


# ----------- ready_to_upload | needs_to_be_added | has_errors  ----------- #
output_df.to_excel(r"{}\{}".format(export_at,"Coldwell ready_to_upload (units).xlsx"), index=False)
has_errors_df.to_excel(r"{}\{}".format(export_at,"Coldwell has_errors.xlsx"), index=False)
needs_to_be_added.to_excel(r"{}\{}".format(export_at,"Coldwell needs_to_be_added (units).xlsx"), index=False)


# ----------- Down Payments|Starting Price + Add percentage format to the "Downpayment" columns  ----------- #
df_to_sheet = {
    "Down_payments_to_upload": [Down_payments_to_upload, "Coldwell ready_to_upload (Down Payments).xlsx", "B"],
    "Down_payments_to_be_added": [Down_payments_to_be_added, "Coldwell needs_to_be_added (Down Payments).xlsx", "B"],
    "starting_price_df_to_upload": [starting_price_df_to_upload, "Coldwell ready_to_upload (Starting Prices).xlsx", "D"],
    "starting_price_df_to_be_added": [starting_price_df_to_be_added, "Coldwell needs_to_be_added (Starting Prices).xlsx", "D"]
}

# Process and export dataframes
for key, value in df_to_sheet.items():
    # Drop duplicates in the dataframe (if not already dropped)
    value[0] = value[0].drop_duplicates()

    # Create the Excel path
    excel_path = export_at + "\\" + value[1]

    # Export the dataframe to Excel
    value[0].to_excel(excel_path, index=False)


total == needs_to_be_added.shape[0] + has_errors_df.shape[0] + output_df.shape[0]